# Tutorial: Discrete Dynamic Programming (2)

## Advanced Macro: numerical methods

### Job-Search Model

- When unemployed in date, a job-seeker
  - consumes unemployment benefit $c_t = \underline{c}$
  - receives in every date $t$ a job offer $w_t$
    - $w_t$ is i.i.d., 
    - takes values $w_1, w_2, w_3$ with probabilities $p_1, p_2, p_3$
  - if job-seeker accepts, becomes employed at rate $w_t$ in the next period
  - else he stays unemployed
  
- When employed at rate $w$
  - worker consumes salary $c_t = w$
  - with small probability $\lambda>0$ looses his job:
    - starts next period unemployed
  - otherwise stays employed at same rate
- Objective: $\max E_0 \left\{ \sum \beta^t \log(w_t) \right\}$


__What are the states, the controls, the reward of this problem ? Write down the Bellman equation.__

Here is the Bellman equation:

- If Unemployed, facing current offer $w$:  
$$V^U(w) = U(\underline{c}) + \max_{a} \begin{cases} \beta V^E(w) & \text{if $a(w)$ is true} \\\\ \beta  \\E_{w'}\left[ V^U(w^{\prime}) \right]  & \text{if $a(w)$ is false} \end{cases}$$
- If Employed, at rate $w$
$$V^E(w) = U(w) +  (1-\lambda) \beta V^E(w) +  \lambda \beta \\E_{w'}\left[ V^U(w^{\prime}) \right] $$


__Define a parameter structure for the model.__

In [2]:
struct Parameters
    β::Float64
    cbar::Float64
    λ::Float64
    w::Vector{Float64}
    p::Vector{Float64}
end

In [4]:
p = Parameters(0.9, 0.5, 0.1, [0.9,1,1.1], [0.3,0.4,0.3])

Parameters(0.9, 0.5, 0.1, [0.9, 1.0, 1.1], [0.3, 0.4, 0.3])

__Define a function  `value_update(V_U::Vector{Float64}, V_E::Vector{Float64}, x::Vector{Bool}, p::Parameters)::Tuple{Vector, Vector}`, which takes in value functions tomorrow and a policy vector and return updated values for today.__

In [5]:
function value_update(V_U, V_E, x, p)
    
    V_U_0 = zeros(3)
    V_E_0 = zeros(3)
    
    # value being unemployed
    for i=1:3
#         w_0 = p.w # job offer
        U_0 = log(p.cbar)
        if x[i] # if offer is accepted
            V_U_0[i] = U_0  + p.β * V_E[i]
        else
            E_V_U = sum( p.p[j]*V_U[j] for j=1:3 ) # expected value of being unemployed tomorrow
            V_U_0[i] = U_0 + p.β * E_V_U
        end
    end
    
    # value being employed
    E_V_U = sum( p.p[j]*V_U[j] for j=1:3 )  # expected value of being unemployed tomorrow
    for i=1:3
        U_0 = log(p.w[i])
        V_E_0[i] = U_0 + p.β*( (1-p.λ)*V_E[i] + p.λ* E_V_U   )
    end
    

    return V_U_0, V_E_0
end

value_update (generic function with 1 method)

In [7]:
V0 = zeros(3)
x0 = [false, false, true]
r = value_update(V0, V0, x0, p)

([-0.6931471805599453, -0.6931471805599453, -0.6931471805599453], [-0.10536051565782628, 0.0, 0.09531017980432493])

__Define a function __Define a function `policy_eval(x::Vector{Bool}, p::Parameter)::Tuple{Vector, Vector}` which takes in a policy vector and returns the value(s) of following this policies forever. You can add relevant arguments to the function.__ which takes in a policy vector and returns the value(s) of following this policies forever. You can add relevant arguments to the function.__

In [23]:
norm(u::Tuple{Vector, Vector}) = max( maximum(abs,u[1]), maximum(abs,u[2]))

norm (generic function with 2 methods)

In [26]:
import Base.-
-(u::Tuple{Vector, Vector}, v::Tuple{Vector, Vector}) = (u[1]-v[1], u[2]-v[2] )

- (generic function with 193 methods)

In [43]:
function policy_eval(x, p::Parameters; N=1000, verbose=false, η_tol= 1e-6)
    VU = zeros(3)
    VE = zeros(3)
    η0 = 1
    for n in 1:N
        VU_new, VE_new = value_update(VU, VE, x, p)
        η = norm( (VU_new, VE_new) - (VU, VE) ) # successive approximation errors
        
        if η<η_tol
            return VU_new, VE_new
        end
        
        λ = η/η0
        η0 = η
        if verbose
            println(n, " : ", η, " : ", λ)
        end
        VU,VE = VU_new,VE_new
    end
    return VU, VE
    
end

policy_eval (generic function with 1 method)

In [45]:
policy_eval(x0, p, verbose=false)

([-2.766351611958541, -2.766351611958541, -1.2237176460284465], [-1.6456852372477617, -1.0911562075082548, -0.5895236822571854])

__Define a function `bellman_step(V_E::Vector, V_U::Vector, p::Parameters)::Tuple{Vector, Vector, Vector}` which returns updated values, together with improved policy rules.__

In [47]:
function bellman_update(V_U, V_E, p)
    
    V_U_0 = zeros(3)
    V_E_0 = zeros(3)
    x = zeros(Bool, 3)
    
    # value being unemployed
    for i=1:3
#         w_0 = p.w # job offer
        U_0 = log(p.cbar)
        
        # value of acceppting
        Vaccept =  U_0  + p.β * V_E[i]
        
        # value of rejecting
        E_V_U = sum( p.p[j]*V_U[j] for j=1:3 ) # expected value of being unemployed tomorrow
        Vreject = U_0 + p.β * E_V_U
        
        if Vaccept>Vreject
            V_U_0[i] = Vaccept
            x[i] = true
        else
            V_U_0[i] = Vreject
            x[i] = false
        end

    end
    
    # value being employed
    E_V_U = sum( p.p[j]*V_U[j] for j=1:3 )  # expected value of being unemployed tomorrow
    for i=1:3
        U_0 = log(p.w[i])
        V_E_0[i] = U_0 + p.β*( (1-p.λ)*V_E[i] + p.λ* E_V_U   )
    end
    

    return V_U_0, V_E_0, x
end

bellman_update (generic function with 1 method)

In [50]:
bellman_update(V0, V0, p)

([-0.6931471805599453, -0.6931471805599453, -0.6931471805599453], [-0.10536051565782628, 0.0, 0.09531017980432493], Bool[0, 0, 0])

__Implement Value Function Iteration__

In [74]:
function vfi(p::Parameters; N=1000, η_tol=1e-6, verbose=true)
    VU = zeros(3)
    VE = zeros(3)
    x0 = zeros(Bool, 3)
    η0 = 1.0
    for n=1:N
        
        VUn, VEn, x = bellman_update(VU, VE, p)
        
        
        eval
        η = norm( (VUn, VEn) - (VU, VE) ) # successive approximation errors
    
        if η<η_tol
            return VUn, VEn, x
        end
    
        λ = η/η0
        η0 = η
        if verbose
            println(n, " : ", η, " : ", λ)
        end
        
        VU, VE = VUn, VEn
    end
end

vfi (generic function with 4 methods)

In [77]:
p0 = Parameters(0.9, 0.5, 0.1, [0.9,1,1.1], [0.3,0.4,0.3])


Parameters(0.9, 0.5, 0.1, [0.9, 1.0, 1.1], [0.3, 0.4, 0.3])

In [75]:
vfi(p; verbose=false)

([-1.7179192488891877, -1.2188431221884017, -0.7673738495210309], [-1.138636559674094, -0.5841075299855604, -0.08247500478060171], Bool[1, 1, 1])

In [92]:
p1 = Parameters(0.9, 0.6, 0.1, [0.9,1,1.1], [0.3,0.4,0.3])
vfi(p1; verbose=false)

([-1.2936319663132243, -0.8816242128292631, -0.43015494039044955], [-0.96652839387281, -0.4119993644116685, 0.08963316058758863], Bool[0, 1, 1])

__Implement Policy Iteration and compare rates of convergence.__

In [100]:
function policy_iteration(p::Parameters; N=1000, η_tol=1e-6, verbose=true)
    VU = zeros(3)
    VE = zeros(3)
    x = zeros(Bool, 3)
    η0 = 1.0
    for n=1:N
        
        println("policy: ", x)
        
        VUn, VEn, x = bellman_update(VU, VE, p)
        VUnn, VEnn = policy_eval(x, p)
        
        η = norm( (VUnn, VEnn) - (VU, VE) ) # successive approximation errors
    

        λ = η/η0
        η0 = η
        if verbose
            println(n, " : ", η, " : ", λ)
        end
        
        if η<η_tol
            return VUnn, VEnn, x
        end
    
        
        VU, VE = VUnn, VEnn
    end
end

policy_iteration (generic function with 1 method)

In [101]:
policy_iteration(p1)

policy: Bool[0, 0, 0]
1 : 5.108247471106895 : 5.108247471106895
policy: Bool[0, 0, 0]
2 : 4.658682056690526 : 0.9119922405953926
policy: Bool[1, 1, 1]
3 : 0.10647875505810611 : 0.02285598239209898
policy: Bool[0, 1, 1]
4 : 0.0 : 0.0


([-1.293632058559428, -0.8816243050039159, -0.43015503273205447], [-0.9665284858885164, -0.41199945659347637, 0.08963306825552382], Bool[0, 1, 1])

__Discuss the Effects of the Parameters__

### Simulating an AR1

Choose a 2x2 matrix $P$ (with spectral radius <1) and a 2x2 matrix Q.

Consider the VAR1 process $x_t = P x_{t-1} + Q \epsilon_t$ where $\epsilon_t= (\eta_{1,t}, \eta_{2,t})$ with $\eta_1\sim\mathcal{N}(0,1)$ and $\eta_1\sim\mathcal{N}(0,1)$

__Compute impulse response functions.__

__Simulate the process for $T$ periods.__

__Simulate the process $N=1000$ times for $T=1000$ periods. How would you store the results?__

__Make density plots to illustrate the ergodic property of the process__

__Compute the asymptotic variance of the process. Compare with the theoretical one.__